In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("insurance_pre.csv")
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [3]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [4]:
independent=dataset[['age', 'bmi', 'children', 'sex_male', 'smoker_yes']]
independent

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,0,1
1,18,33.770,1,1,0
2,28,33.000,3,1,0
3,33,22.705,0,1,0
4,32,28.880,0,1,0
...,...,...,...,...,...
1333,50,30.970,3,1,0
1334,18,31.920,0,0,0
1335,18,36.850,0,0,0
1336,21,25.800,0,0,0


In [5]:
dependent=dataset[['charges']]
dependent

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(independent,dependent, test_size=0.30,random_state=0)

In [13]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
Param_grid={'criterion':['squared_error','absolute_error','friedman_mse','poisson'],'max_features':['sqrt','log2'],'n_estimators':[10,100]}
grid=GridSearchCV(RandomForestRegressor(), Param_grid, refit = True, verbose = 3,n_jobs=-1)
grid.fit(X_train,Y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


C:\Users\Raji\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


GridSearchCV(estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'criterion': ['squared_error', 'absolute_error',
                                       'friedman_mse', 'poisson'],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [10, 100]},
             verbose=3)

In [21]:
re=grid.cv_results_
grid_predictions = grid.predict(X_test)
from sklearn.metrics import r2_score
r_score=r2_score(Y_test,grid_predictions)
print("The R_score value for best parameter {}:".format(grid.best_params_),r_score)

The R_score value for best parameter {'criterion': 'poisson', 'max_features': 'log2', 'n_estimators': 100}: 0.8724658021625629


In [23]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.032251,0.004086,0.004064,0.002076,squared_error,sqrt,10,"{'criterion': 'squared_error', 'max_features':...",0.836469,0.785850,0.799809,0.817986,0.767662,0.801555,0.024031,10
1,0.217880,0.007059,0.010632,0.000392,squared_error,sqrt,100,"{'criterion': 'squared_error', 'max_features':...",0.863527,0.785899,0.808469,0.822901,0.772075,0.810574,0.031785,6
2,0.026074,0.001935,0.003770,0.001210,squared_error,log2,10,"{'criterion': 'squared_error', 'max_features':...",0.848848,0.767882,0.786945,0.795770,0.768819,0.793653,0.029587,13
3,0.215582,0.011840,0.009406,0.001450,squared_error,log2,100,"{'criterion': 'squared_error', 'max_features':...",0.863878,0.794096,0.806739,0.828115,0.765449,0.811655,0.033062,5
4,0.060644,0.004085,0.003611,0.001502,absolute_error,sqrt,10,"{'criterion': 'absolute_error', 'max_features'...",0.857035,0.786941,0.800030,0.790891,0.769533,0.800886,0.029769,11
5,0.509411,0.003490,0.009171,0.001125,absolute_error,sqrt,100,"{'criterion': 'absolute_error', 'max_features'...",0.857540,0.791382,0.805432,0.821075,0.772661,0.809618,0.028788,8
6,0.052438,0.001155,0.003213,0.000985,absolute_error,log2,10,"{'criterion': 'absolute_error', 'max_features'...",0.830048,0.777809,0.811757,0.804874,0.766953,0.798288,0.022955,12
7,0.532055,0.015511,0.007837,0.000803,absolute_error,log2,100,"{'criterion': 'absolute_error', 'max_features'...",0.863690,0.787189,0.809918,0.830414,0.770447,0.812331,0.032732,3
8,0.026916,0.003504,0.002317,0.000870,friedman_mse,sqrt,10,"{'criterion': 'friedman_mse', 'max_features': ...",0.842564,0.774162,0.775522,0.807265,0.756659,0.791234,0.030416,14
9,0.203455,0.004342,0.008690,0.001156,friedman_mse,sqrt,100,"{'criterion': 'friedman_mse', 'max_features': ...",0.863093,0.793917,0.809087,0.823649,0.771595,0.812268,0.030705,4


In [25]:
age_input=float(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=float(input("Children:"))
sex_male_input=int(input("Sex Male 0 or 1:"))
smoker_yes_input=int(input("Smoker Yes 0 or 1:"))

Age: 56
BMI: 24
Children: 2
Sex Male 0 or 1: 1
Smoker Yes 0 or 1: 0


In [27]:
Future_Prediction=grid.predict([[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[15061.2235978]
